In [1]:
import time
import os
import itertools
import numpy as np
import matplotlib.pyplot as plt

import hoomd
import gsd.hoomd
import freud
import coxeter

from simulation import init, randomize

from verts_lib import octa, hexbp, trunc_octa

In [2]:
# List of fixed parameter
N_particles_arr = [1000]
vertices_arr = [hexbp]
particle = coxeter.shapes.ConvexPolyhedron( vertices_arr[0] )
particle_volume_arr = [particle.volume]
pressure_arr = [12., 14.]

# List to explore
seed_arr = [i for i in range(1,3)]
inertia_arr = []
for ri in np.array([1.]):
    diag_inertia = particle.inertia_tensor.diagonal().copy()
    diag_inertia[2] *= ri
    inertia_arr.append(diag_inertia)
    
sp_dict = list(itertools.product(N_particles_arr, 
                                 vertices_arr, 
                                 particle_volume_arr, 
                                 pressure_arr, 
                                 seed_arr, 
                                 inertia_arr))


In [6]:
class JobDict(dict):
    
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    

class Job:
    
    def __init__(self, job_id, job_path, statepoint):
    
        self.job_id = job_id
        self.job_path = job_path
        
        self.statepoint_path = os.path.join(self.job_path, 'statepoint.npy')
        self.document_path = os.path.join(self.job_path, 'document.npy')
        
        if os.path.exists(self.statepoint_path):
            statepoint = np.load(self.statepoint_path, allow_pickle=True).tolist()
        
        self.sp = JobDict(statepoint)
        self.statepoint = self.sp
        
        if os.path.exists(self.document_path):
            document = np.load(self.document_path, allow_pickle=True).tolist()
        else:
            document = {}
            
        self.doc = JobDict(document)
        self.document = self.doc
        
        
    def init(self):
        
        if not os.path.isdir(self.job_path):
            os.makedirs(self.job_path)
        
        if not os.path.exists(self.statepoint_path):
            np.save(self.statepoint_path, dict(self.sp))
        else:
            print('job already exist!')
            
    def fn(self, filename):
        
        job_associated_path = os.path.join(self.job_path, filename)
        
        return job_associated_path
    
    def update(self):
        
        np.save(self.statepoint_path, dict(self.sp))
        np.save(self.document_path, dict(self.doc))
    
        
class Project:

    def __init__(self, project_name='project'):
        
        self.project_name = project_name
        self.project_path = self.project_name
        self.job_id = -1
        
        self.id2sp = {}
        self.id2job_path = {}
        
        if not os.path.isdir(self.project_path):
            os.makedirs(self.project_path)
            
    def open_job(self, statepoint):
        """
        Open a new job without overwriting existing jobs.
        """
        self.job_id += 1
        job_path = os.path.join(self.project_path, '{:06d}'.format(self.job_id))
        if not os.path.isdir(job_path):
            
            self.id2sp[self.job_id] = statepoint
            self.id2job_path[self.job_id] = job_path
            
            return Job(self.job_id, job_path, statepoint)
        
        else:
            if not self.id2sp or self.id2job_path: 
                self.get_jobs() # run only once
            statepoint_i = self.id2sp[self.job_id]
            job_path_i = self.id2job_path[self.job_id]
            
            return Job(self.job_id, job_path_i, statepoint_i)
        
    def get_jobs(self):
        """
        Get jobs in the project folder.
        """
        for job_id_i in os.listdir(self.project_name):
            
            i = int(job_id_i)
            job_path_i = os.path.join(self.project_name, job_id_i)
            statepoint_path_i = os.path.join(job_path_i, 'statepoint.npy')
            statepoint_i = np.load(statepoint_path_i, allow_pickle=True).tolist()
            
            self.id2sp[i] = statepoint_i
            self.id2job_path[i] = job_path_i
        
    def find_jobs(self, condition=None):
        """
        Find jobs which match the condition.
        """
        if condition==None:
            for job_id_i, job_path_i in self.id2job_path.items():
                statepoint_i = self.id2sp[job_id_i]
                yield Job(job_id_i, job_path_i, statepoint_i)
        
        else:
            for job_id_i, statepoint_i in self.id2sp.items():
                if all(statepoint_i[k] == v for k, v in condition.items()):
                    job_path_i = self.id2job_path[job_id_i]
                    yield Job(job_id_i, job_path_i, statepoint_i)
            
    def find_job_ids(self, job_id_arr):
        """
        Find jobs which match the job_id.
        """
        for job_id_i in job_id_arr:
            job_path_i = self.id2job_path[job_id_i]
            statepoint_i = self.id2sp[job_id_i]
            yield Job(job_id_i, job_path_i, statepoint_i)


In [7]:
project = Project('test_project')
project.get_jobs()

In [8]:
for job in project.find_job_ids([1,2,3]):
    
    print(job.sp)

{'N_particles': 1000, 'vertices': array([[ 0.00000000e+00,  0.00000000e+00,  1.12000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
       [-5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
       [-1.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-5.00000000e-01, -8.66025404e-01,  0.00000000e+00],
       [ 5.00000000e-01, -8.66025404e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.12000000e+00]]), 'particle_volume': 1.9398969044771421, 'pressure': 12.0, 'seed': 2, 'inertia': array([0.48582778, 0.48582778, 0.48497423]), 'new_var': 1}
{'N_particles': 1000, 'vertices': array([[ 0.00000000e+00,  0.00000000e+00,  1.12000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
       [-5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
       [-1.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-5.0

#### Loading the project the first time

In [4]:
project = Project('test_project')

for N_particles, vertices, particle_volume, pressure, seed, inertia in sp_dict:
    statepoint = dict(N_particles=N_particles, 
                      vertices=vertices,
                      particle_volume=particle_volume,
                      pressure=pressure, 
                      seed=seed,
                      inertia=inertia)

    job = project.open_job(statepoint)
    job.init()

#### Add a new variable to the statepoint

In [5]:
for job in project.find_jobs():
    job.sp.new_var=1
    job.update()
    
for job in project.find_jobs():
    print(job.sp.new_var)

1
1
1
1


#### Find jobs matching conditions

In [6]:
condition = {'N_particles': 1000, 'pressure': 12., 'seed': 1}

for job_i in project.find_jobs(condition):
    statepoint0 = np.load(job.fn('statepoint.npy'), allow_pickle=True).tolist()
    print( statepoint0 )
    print('N_particles:', job_i.sp.N_particles)
    print('seed:', job_i.sp.seed)
    print('pressure:', job_i.sp.pressure)
    print('------------------------\n')

{'N_particles': 1000, 'vertices': array([[ 0.00000000e+00,  0.00000000e+00,  1.12000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
       [-5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
       [-1.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-5.00000000e-01, -8.66025404e-01,  0.00000000e+00],
       [ 5.00000000e-01, -8.66025404e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.12000000e+00]]), 'particle_volume': 1.9398969044771421, 'pressure': 14.0, 'seed': 2, 'inertia': array([0.48582778, 0.48582778, 0.48497423]), 'new_var': 1}
N_particles: 1000
seed: 1
pressure: 12.0
------------------------



#### Find all jobs

In [7]:
for job_i in project.find_jobs():
    
    sp_load = np.load(job.fn('statepoint.npy'), allow_pickle=True)
    
    print('N_particles:', job_i.sp.N_particles, 
          '| seed:', job_i.sp.seed, 
          '| pressure:', job_i.sp.pressure)

N_particles: 1000 | seed: 1 | pressure: 12.0
N_particles: 1000 | seed: 2 | pressure: 12.0
N_particles: 1000 | seed: 1 | pressure: 14.0
N_particles: 1000 | seed: 2 | pressure: 14.0


#### Add job document

In [8]:
for job in project.find_jobs():
    
    particle = coxeter.shapes.ConvexPolyhedron( job.sp.vertices )
    job.doc['verts'] = particle.vertices.tolist() 
    job.doc['faces'] = particle.faces 
    job.doc['sigma'] = 2*particle.insphere_from_center.radius 
    job.doc['r_cut'] = 2*particle.circumsphere_from_center.radius + 0.15*job.doc.sigma
    job.update()

#### Get existing project

In [9]:
project = Project('test_project')
project.get_jobs()

In [10]:
for job in project.find_jobs():
    job.doc['init']=False
    job.doc['randomize']=False
    job.update()

In [15]:
for job in project.find_jobs():
    print(job.doc.init)

True
True
True
True


In [14]:
for job in project.find_jobs():
    
    if job.doc['init']==False:
        init(job)
    if job.doc['randomize']==False:
        randomize(job)
        